<hr>

# 🤖 04 - MACHINE LEARNING 🤖

<style>
h1 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

<hr>

### 📂 IMPORTs


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [17]:
# install xgboost if not already installed
#!pip install xgboost

In [18]:
# install shap if not already installed
#!pip install shap

In [19]:
# install numpy and pandas if not already installed
#!pip install numpy pandas

In [20]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from xgboost import XGBClassifier

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)



<hr>

## 📊 STEP 1 — Load Data


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [21]:
import pandas as pd
# load the clean dataset
df = pd.read_csv("../data/processed/online_shoppers_intention_01_standard.csv")

# For consistency with code:
    # X = features without the target variable
    # y = target variable "revenue"

# drop the target variable "revenue" from the features
X = df.drop("revenue", axis=1)

# target variable "revenue" is already binary (0 and 1) 
y=df["revenue"]

# if it's not already in that format (it should be, but just to be safe)
# y = df["revenue"].astype(int)


In [22]:
df_X_train = pd.read_csv("../data/processed/X_train.csv")
df_X_train.head()

,admin,admin_duration,info,info_duration,prod_related,prod_related_duration,bounce_rate,exit_rate,page_value,special_day,month,os,browser,region,traffic_type,visitor_type,weekend
0,8,732.25,0,0.0,34,1345.755952,0.005128,0.013342,12.274195,0.8,May,3,3,3,2,Returning_Visitor,False
1,0,0.00,0,0.0,4,157.200000,0.040000,0.100000,0.000000,0.0,Oct,1,8,3,1,Returning_Visitor,True
2,0,0.00,0,0.0,4,42.000000,0.000000,0.050000,0.000000,0.0,Dec,2,10,1,2,Returning_Visitor,False
3,2,338.00,0,0.0,17,1205.566667,0.012500,0.037500,19.236250,0.8,May,3,2,2,2,Returning_Visitor,False
4,0,0.00,0,0.0,32,827.646212,0.000000,0.000587,54.676348,0.0,Mar,2,2,7,2,Returning_Visitor,False


<hr>

## ✂️ STEP 2 — Train/Test Split


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [23]:
# Split the data into:
# - Training set
# - Test set
# Using stratify to maintain class balance

(
    X_train,     # features for training
    X_test,      # features for testing
    y_train,       # target for training
    y_test         # target for testing
) = train_test_split(
    X, # features without the target variable "revenue"
    y, # target variable "revenue"
    test_size=0.2,      # 20% test set, 80% train set
    stratify=y,         # maintain class balance in splits
    random_state=42     # for reproducibility, we can choose any integer as the random state
)





<hr>

## 🧹 STEP 3 — Preprocessing


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

### ML (ENCODE CATEGORICAL COLUMNS)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Define categorical column names properly as a list
categorical_cols = [
    "month",
    "visitor_type",
    "browser",
    "os",
    "region",
    "traffic_type"
]

# Initialize encoder
encoder = OneHotEncoder(sparse_output=False)

# Encode categorical columns
encoded = encoder.fit_transform(df[categorical_cols])

encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(categorical_cols),
    index=df.index
)

# Drop original categorical columns
df_coded = df.drop(columns=categorical_cols)

# Add encoded columns
df_coded = pd.concat([df_coded, encoded_df], axis=1)

# Convert binary columns to 0/1
df_coded["weekend"] = df_coded["weekend"].astype(int)
df_coded["revenue"] = df_coded["revenue"].astype(int)

# Save to CSV
df_coded.to_csv("../data/processed/online_shoppers_intention_07_encoded.csv",index=False)




# load the encoded dataset in DataFrame
df = pd.read_csv("../data/processed/online_shoppers_intention_07_encoded.csv")

[X_train] saved to CSV file.
[X_test] saved to CSV file.
[y_train] saved to CSV file.
[y_test] saved to CSV file.


In [2]:
#Nominal columns
nominal_cols = ["month", "visitor_type", "browser", "os", "region", "traffic_type"]

X_train = X_train[nominal_cols]
X_test  = X_test[nominal_cols]

#Numerical columns
X_train = X_train.drop(columns=nominal_cols)
X_test  = X_test.drop(columns=nominal_cols)

#OneHotEncoder
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False) #for safe handling unseen categories in test set, and to get an array output
ohe.fit(X_train)

X_train_enc = ohe.transform(X_train)
X_test_enc  = ohe.transform(X_test)

#Back to DataFrame
encoded_cols = ohe.get_feature_names_out(nominal_cols) #to create new column names for the encoded features

X_train_df = pd.DataFrame(X_train_enc,
                              columns=encoded_cols,
                              index=X_train.index)

X_test_df = pd.DataFrame(X_test_enc,
                             columns=encoded_cols,
                             index=X_test.index)

#Concat
X_train_final = pd.concat([X_train, X_train_df], axis=1)
X_test_final  = pd.concat([X_test, X_test_df], axis=1)



# Save to CSV
X_train_final.to_csv("../data/processed/X_train.csv", index=False)
X_test_final.to_csv("../data/processed/X_test.csv", index=False)

y_train.to_csv("../data/processed/y_train.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)

NameError: name 'X_train' is not defined

In [25]:
df_X_train = pd.read_csv("../data/processed/X_train.csv")
df_X_train.head()

,admin,admin_duration,info,info_duration,prod_related,prod_related_duration,bounce_rate,exit_rate,page_value,special_day,month,os,browser,region,traffic_type,visitor_type,weekend
0,8,732.25,0,0.0,34,1345.755952,0.005128,0.013342,12.274195,0.8,May,3,3,3,2,Returning_Visitor,False
1,0,0.00,0,0.0,4,157.200000,0.040000,0.100000,0.000000,0.0,Oct,1,8,3,1,Returning_Visitor,True
2,0,0.00,0,0.0,4,42.000000,0.000000,0.050000,0.000000,0.0,Dec,2,10,1,2,Returning_Visitor,False
3,2,338.00,0,0.0,17,1205.566667,0.012500,0.037500,19.236250,0.8,May,3,2,2,2,Returning_Visitor,False
4,0,0.00,0,0.0,32,827.646212,0.000000,0.000587,54.676348,0.0,Mar,2,2,7,2,Returning_Visitor,False


<hr>

## 📊 STEP 4 — DISTIRBUTION ENTIRE/TRAIN/TEST


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [26]:
# For consistency in code team collaboration 
# assign:  X_train,  X_test,  y_train,  y_test

print(70*"-")

# calculate the class imbalance ratio for the training set to use in XGBoost's scale_pos_weight parameter
ratio = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Class imbalance ratio (negative:positive) in training set: {ratio:.2f}")
    # This ratio will be used in the XGBoost model to help it handle the class imbalance effectively.
    # A ratio greater than 1 indicates that the negative class (0) is more prevalent than the positive class (1),  which is common in many real-world datasets. 
    # By setting scale_pos_weight to this ratio, we can help XGBoost give more attention to the minority class during training.

print(70*"-")

# distribution ENTIRE dataset
print("Class distribution in entire dataset:")
print(pd.concat([y_train, y_test]).value_counts())
# percentage ENTIRE dataset
print("\nClass distribution percentages in entire dataset:")
print(pd.concat([y_train, y_test]).value_counts(normalize=True) * 100)
print(70*"-")

# distribution TRAIN set
print("Class distribution in training set:")
print(y_train.value_counts())
# percentage TRAIN set
print("\nClass distribution percentages in training set:")
print(y_train.value_counts(normalize=True) * 100)
print(70*"-")


# distribution TEST set
print("Class distribution in test set:")
print(y_test.value_counts())
# percentage TEST set
print("/nClass distribution percentages in test set:")
print(y_test.value_counts(normalize=True) * 100)
print(70*"-")


----------------------------------------------------------------------
Class imbalance ratio (negative:positive) in training set: 5.46
----------------------------------------------------------------------
Class distribution in entire dataset:
revenue
False    10422
True      1908
Name: count, dtype: int64

Class distribution percentages in entire dataset:
revenue
False    84.525547
True     15.474453
Name: proportion, dtype: float64
----------------------------------------------------------------------
Class distribution in training set:
revenue
False    8338
True     1526
Name: count, dtype: int64

Class distribution percentages in training set:
revenue
False    84.529603
True     15.470397
Name: proportion, dtype: float64
----------------------------------------------------------------------
Class distribution in test set:
revenue
False    2084
True      382
Name: count, dtype: int64
/nClass distribution percentages in test set:
revenue
False    84.509327
True     15.490673
Name: pr

<hr>

## 📐 STEP 5 — ML MODEL RESULTS


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
import pandas as pd
X_train = pd.read_csv("../data/processed/X_train.csv")
X_test = pd.read_csv("../data/processed/X_test.csv")   
y_train = pd.read_csv("../data/processed/y_train.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")

In [27]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    matthews_corrcoef, confusion_matrix
)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from xgboost import XGBClassifier


# 1️⃣ Define Feature Groups

numeric_features = [
    "admin", "admin_duration",
    "info", "info_duration",
    "prod_related", "prod_related_duration",
    "bounce_rate", "exit_rate",
    "page_value", "special_day"
]

# All remaining columns except target are binary (0/1)
binary_features = [col for col in X.columns if col not in numeric_features]


# 2️⃣ Define Preprocessors

# For linear models & KNN (need scaling)
preprocessor_scaled = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("bin", "passthrough", binary_features)
    ]
)

# For tree-based models (no scaling needed)
preprocessor_tree = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("bin", "passthrough", binary_features)
    ]
)


# 3️⃣ Class Imbalance Ratio

ratio = (y == 0).sum() / (y == 1).sum()


# 4️⃣ Define Models

models = {

    "Logistic Regression": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", LogisticRegression(
            class_weight='balanced',
            max_iter=5000,
            solver="lbfgs"
        ))
    ]),

    "Linear SVM": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", SVC(
            kernel='linear',
            class_weight='balanced',
            probability=True
        ))
    ]),

    "KNN": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", KNeighborsClassifier())
    ]),

    "Decision Tree": Pipeline([
        ("prep", preprocessor_tree),
        ("model", DecisionTreeClassifier(
            class_weight='balanced',
            random_state=42
        ))
    ]),

    "Random Forest": Pipeline([
        ("prep", preprocessor_tree),
        ("model", RandomForestClassifier(
            class_weight='balanced',
            random_state=42
        ))
    ]),

    "XGBoost": Pipeline([
        ("prep", preprocessor_tree),
        ("model", XGBClassifier(
            scale_pos_weight=ratio,
            eval_metric='logloss',
            use_label_encoder=False,
            random_state=42
        ))
    ]),

    "AdaBoost": Pipeline([
        ("prep", preprocessor_tree),
        ("model", AdaBoostClassifier(
            random_state=42
        ))
    ])
}


# 5️⃣ Cross Validation Setup

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []


# 6️⃣ Model Evaluation Loop

for name, model in models.items():

    precision_list = []
    recall_list = []
    f1_list = []
    roc_list = []
    pr_list = []
    mcc_list = []

    tp_total = tn_total = fp_total = fn_total = 0

    for train_idx, test_idx in cv.split(X, y):

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        precision_list.append(precision_score(y_test, y_pred))
        recall_list.append(recall_score(y_test, y_pred))
        f1_list.append(f1_score(y_test, y_pred))
        roc_list.append(roc_auc_score(y_test, y_proba))
        pr_list.append(average_precision_score(y_test, y_proba))
        mcc_list.append(matthews_corrcoef(y_test, y_pred))

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        tp_total += tp
        tn_total += tn
        fp_total += fp
        fn_total += fn


    results.append([
        name,
        np.mean(precision_list),
        np.mean(recall_list),
        np.mean(f1_list),
        np.sum(y == 1),
        np.mean(pr_list),
        np.mean(roc_list),
        np.mean(f1_list),
        np.std(f1_list),
        np.mean(mcc_list),
        tp_total, tn_total, fp_total, fn_total
    ])


# 7️⃣ Create Results Table

columns = [
    "Model", "Precision", "Recall", "F1 Score",
    "Support", "PR-AUC", "ROC-AUC",
    "Mean (F1)", "Std (F1)",
    "MCC", "TP", "TN", "FP", "FN"
]

results_df = pd.DataFrame(results, columns=columns)

print(results_df.sort_values(by="F1 Score", ascending=False))

ValueError: could not convert string to float: 'Feb'

<hr>

## 🏆 STEP 6 —  Select TOP 3 Models


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### MODEL 1:


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

# 🏁 Final Model Selection Criteria

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

Choose model with:
- Highest CV ROC-AUC
- Good F1
- Good Recall (important for purchase detection)
- Stable performance (low variance across folds)

<hr>

# 📊 PLOTS

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

<hr>

## 01 - ROC Curves (Compare top 3 models)


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
# paste the plot version you want to add to main notebook here

<hr>

### 01 - ROC Curves - Model 1


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 01 - ROC Curves - Model 2


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 01 - ROC Curves - Model 3


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

## 02 - Adjust Decision Threshold


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- **Instead of 0.5:**
- **This should improve recall.**

In [ ]:
# paste the plot version you want to add to main notebook here

<hr>

### 02 -Adjust Decision Threshold - V1


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 02 - Adjust Decision Threshold - V2


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: pink;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

## 03 - Feature Importance


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- **For RF / XGBoost: Use  `.feature_importances_`**

In [ ]:
import matplotlib.pyplot as plt